# OpenDC

Data centers are becoming an increasingly large contributor to the global carbon footprint. However, because of a lack of tools / guidelines, it has been challenging to optimize data centers for carbon emissions. This is amplified by the fact that running experiments on data centers is both expansive and time consuming. 

OpenDC is a event-based discrete data center simulator. Using such a tool we can do experiments on data centers in a cost-effective and flexible way. In this demo we are going to compare a small and large data center using OpenDC. We are comparing the data centers in terms of performance and sustainability.

You can read more about OpenDC [here](https://opendc.org/).

# 1. Topology

To run a simulation, OpenDC needs a definition of the data center, which we call the topology. The topology of a data center can influence its performance and sustainability greatly. It determines which tasks can be run, how efficiently they are run, and if they can be executed in parallel.

The topology of datacenter is provided using a JSON file. This file defines how many clusters are available, the hosts they contain, and what type of hosts they are. In this demo we compare the performance of two topologies: [small](topologies/1.demo_basic/large_topology.json) and [here](topologies/1.demo_basic/large_topology.json) which can both be found in the resources/topologies folder. The small topology is shown below

```json
{
    "clusters": [
        {
            "name": "C01",
            "hosts": [
                {
                    "name": "H01",
                    "count": 1,
                    "cpu": {
                        "coreCount": 16,
                        "coreSpeed": 3300
                    },
                    "memory": {
                        "memorySize": 140457600000
                    }
                }
            ]
        }
    ]
}
```

The small data center contains only a single cluster *C01*, which contains a single host *H01*. Host *H01* constains two CPUs with 16 cores running at 3200 Mhz, and has a memory of 140457600000 Bytes.

*Exercise 1:* Can you find out what hardware is contained in the large topology located [here](topologies/1.demo_basic/large_topology.json)?

# 2. Workloads

To run a simulation, OpenDC requires information on the type of workload to execute.


The workload is provided using two files:
<ul>
    <li> **tasks.parquet** provides a general overview of the tasks executed during the workload. </li>
    <li> **fragments.parquet** provides detailed information of each task during its runtime. </li>
</ul>

In this demo we are using running the [bitbrains-small](resources/workloads/bitbrains-small/) dataset as our workload. The bitbrains-small workload consist of 50 tasks with a runtime ranging from less than 7 days to more than 30 days. 

##### Lets have a look at the files

In [ ]:
import pandas as pd

df_tasks = pd.read_parquet("workload_traces/bitbrains-small/tasks.parquet")
df_fragments = pd.read_parquet("workload_traces/bitbrains-small/fragments.parquet")

In [ ]:
df_tasks.head()

In [ ]:
df_fragments.head()

# 3. Carbon Trace

- Carbon Traces define the Carbon Intenisty of the available energy over time

- Gathered using ENTSO-E

- Specific to the location of the data center

- Defined as a JSON file

In [ ]:
df_carbon = pd.read_parquet("carbon_traces/carbon_2022.parquet")

df_carbon.head()

# 4. Experiment

Finally, OpenDC needs an experiment file. The Experiment file describes what needs to be run, how, and when. An experiment is delilvered using a JSON file. The experiment we will use for this demo can be found [here](resources/experiments/1.demo_basic.json) and is shown below:

Example Experiment:
```json
{
    "name": "1.demo_basic",
    "topologies": [
        {
            "pathToFile": "topologies/1.demo_basic/small_topology.json"
        },
        {
            "pathToFile": "topologies/1.demo_basic/large_topology.json"
        }
    ],
    "workloads": [
        {
            "pathToFile": "workload_traces/bitbrains-small",
            "type": "ComputeWorkload"
        }
    ],
    "exportModels": [
        {
            "exportInterval": 30000
        }
    ]
}
```

The scenario file used in this demo defines four variables:
- "name" defines where the ouput files will be stored
- "topologies" defines the different topologies that will be used in the experiments
- "workloads" defines what workloads will be run
- "exportModels" defines how frequent OpenDC should export data


Note: most of the variables in the scenario file are provided as lists. This makes it possible to run different experiments using the same scenario. Graph Greenifier will run all combinations of variables as seperate experiments. In this demo, we providee two topologies, and a single workload. This means that the same workload will be executed on the both scenarios, resulting in two results.

# 5. Running an Experiment

An experiment can be run directly from the terminal using the OpenDCExperimentRunner.

In [ ]:
import subprocess

pathToScenario = "experiments/1.demo_basic.json"
subprocess.run(["OpenDCExperimentRunner/bin/OpenDCExperimentRunner", "--experiment-path", pathToScenario])

## 6. Output

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df_host_small = pd.read_parquet("output/1.demo_basic/raw-output/0/seed=0/host.parquet")
df_power_small = pd.read_parquet("output/1.demo_basic/raw-output/0/seed=0/powerSource.parquet")
df_task_small = pd.read_parquet("output/1.demo_basic/raw-output/0/seed=0/task.parquet")
df_service_small = pd.read_parquet("output/1.demo_basic/raw-output/0/seed=0/service.parquet")

df_host_large = pd.read_parquet("output/1.demo_basic/raw-output/1/seed=0/host.parquet")
df_power_large = pd.read_parquet("output/1.demo_basic/raw-output/1/seed=0/powerSource.parquet")
df_task_large = pd.read_parquet("output/1.demo_basic/raw-output/1/seed=0/task.parquet")
df_service_large = pd.read_parquet("output/1.demo_basic/raw-output/1/seed=0/service.parquet")

### Host
- Information about the host at each timestamp. 
- Examples of metrics: 
    - cpu_utilization
    - power_draw 
    - energy_usage 

In [ ]:
print(f"The host file contains the following columns:\n {np.array(df_host_small.columns)}\n")
print(f"The host file consist of {len(df_host_small)} samples")
df_host_small.head()

### Tasks
- The task file contains all information about the different tasks at each timestamp. 
- Example use cases:
    - when is a task run
    - How long did it take
    - on which host was a task executed

In [ ]:
print(f"The task file contains the following columns:\n {np.array(df_task_small.columns)}")
print(f"The task file consist of {len(df_task_small)} samples")
df_task_small.head()

### Power
- The task file contains all information about the power sources at each timestamp. 
- Example use cases:
    - What is the total energy used during the workload?

In [ ]:
print(f"The task file contains the following columns:\n {np.array(df_power_small.columns)}")
print(f"The power file consist of {len(df_power_small)} samples")
df_power_small.head()

### Service

- The service file contains genaral information about the experiments. 
- Example uses:
    - How many tasks are running?
    - How many hosts are up?

In [ ]:
print(f"The service file contains the following columns:\n {np.array(df_service_small.columns)}")
print(f"The service file consist of {len(df_service_small)} samples")
df_service_small.head()

## 7. Performance

- To properly compare the different experiments, we would like to aggregate them into meaningfull values.

In [ ]:
runtime_small = pd.to_timedelta(df_service_small.timestamp.max() - df_service_small.timestamp.min(), unit="ms")
runtime_large = pd.to_timedelta(df_service_large.timestamp.max() - df_service_large.timestamp.min(), unit="ms")

print(f"The small data center had a total runtime of {runtime_small}")
print(f"The large data center had a total runtime of {runtime_large}\n")



utilization_small = df_host_small.cpu_utilization.mean()
utilization_large = df_host_large.cpu_utilization.mean()

print(f"On average, the utilization of each host in the small data center is {utilization_small * 100:.2f}%")
print(f"On average, the utilization of each host in the large data center is {utilization_large * 100:.2f}%")

## 7. Sustainability

We can also compare the two data centers in terms of sustainabilty

Next we print the total energy usage of the two data centers

In [ ]:
energy_small = df_power_small.energy_usage.sum() / 3_600_000 # convert energy to kWh
energy_large = df_power_large.energy_usage.sum() / 3_600_000 # convert energy to kWh


print(f"The small data center used {energy_small:.2f} kWh during the workload")
print(f"The large data center used {energy_large:.2f} kWh during the workload")

#### Can you think of more metrics to print?

In [ ]:
# Your code here

## 8. Visualization

While single numbers can be great to compare the different workloads, it does not always indicate the reasons for the difference. 

Similarly to the value aggregation, vizualization can be done directly using the pandas dataframes. 
However, Graph Greenifier also provides several predefined plotting tools to help this process.

### Service

Lets start with plotting general information using the service output file. This can be done using the *plotService* function. 
Below, we plot the number of active servers during the workload. 

In [ ]:
plt.plot(df_service_small.tasks_active, label="small")
plt.plot(df_service_large.tasks_active, label="big")


plt.title("active tasks during a workload")
plt.legend()
plt.show()

### Hosts

We can also look at the performance of the hosts. 

Lets plot the utilization of the hosts over time.

In [ ]:
def plotHost(df_host, column, aggregation_method, label, window_size=1000):
    if aggregation_method not in ["mean", "sum"]:
        raise ValueError(f"incorrect aggregation method provided: {aggregation_method}, please pick on of [mean, sum]")

    df_agg = df_host.groupby("timestamp")[[column]].agg(aggregation_method)

    plt.plot(df_agg.index/1000/60/60, df_agg.rolling(window_size, min_periods=1).mean(), label=label)
    plt.xlabel("timestamp (h)")
    plt.ylabel(column)

    plt.legend()


plotHost(df_host_small, "cpu_utilization", "mean", "small")
plotHost(df_host_large, "cpu_utilization", "mean", "large")

### Sustainability

We can also plot sustainability related metrics.

Lets plot the energy usage over time

In [ ]:
plotHost(df_power_small, "energy_usage", "sum", "small")
plotHost(df_power_large, "energy_usage", "sum", "large")

#### Can you think of other metrics to plot? (Maybe Carbon?)

In [ ]:
# Your code here